In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import tensorflow.keras as K
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as bk
# import tensorflow_addons as tfa

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pickle
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# ! cp './drive/MyDrive/Wipro/Data/memotion1_images_combined.zip' .
# ! unzip -q memotion1_images_combined.zip
# ! rm -rf memotion1_images_combined.zip

! cp './drive/MyDrive/Wipro/Final/Data/Memotion/images.zip' .
! unzip -q images.zip
! rm -rf images.zip

In [ ]:
# ! cp './drive/MyDrive/Wipro/Final/Data/Memotion/pickles/memotion_img_embedding.pkl' .
# ! cp './drive/MyDrive/Wipro/Final/Data/Memotion/pickles/memotion_text_embedding.pkl' .
! cp './drive/MyDrive/Wipro/Final/Data/Memotion/pickles/memotion_img_sentence_encodings.pkl' .

## Data Preparation

### Word Level Data

### Extra 

In [ ]:
df_merged = pd.read_csv('./drive/MyDrive/Wipro/Data/merged_data.csv')

In [ ]:
df_merged.head()

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,tagged_sentence
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,positive,look friend lightyear sohalikut trend play 10 ...
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,positive,best 10 yearchallenge
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,sam thorne @ strippin follow follow saw everyo...
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,10 year challenge sweet dee edition
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,10 year challenge filter 47 hilarious 10 year ...


In [ ]:
df_word = pd.read_csv('./drive/MyDrive/Wipro/Data/image_word_pairs.csv')

In [ ]:
df_word.head()

In [ ]:
df_merged.shape,df_word.shape

In [ ]:
df_merged['image_name'].unique().shape

In [ ]:
df_word['image_name'].unique().shape

In [ ]:



rem_images = ['image_1567.jpg','image_4924.jpg','image_5119.png','image_6357.jpg']
df_word_cleaned = df_word.drop(df_word[df_word['image_name'].isin(rem_images)].index)

# df['overall_sentiment'] = df['overall_sentiment'].apply(get_labels)

df_word_cleaned.dropna(inplace=True)

In [ ]:
df_word_cleaned.shape

In [ ]:
df_word_cleaned['image_name'].unique().shape

In [ ]:
def apply_humour(label):

  if label == 'not_funny':
    return 0

  elif label == 'funny':
    return 1

  elif label == 'very_funny':
    return 2

  else: 
    return 3


In [ ]:
def apply_sarcasm(label):

  if label == 'not_sarcastic':
    return 0

  elif label == 'general':
    return 1

  elif label == 'twisted_meaning':
    return 2
    
  else:
    return 3
    

In [ ]:
def apply_offensive(label):

  if label == 'not_offensive':
    return 0

  elif label == 'slight':
    return 1

  elif label == 'very_offensive':
    return 2
    
  else:
    return 3


In [ ]:
def apply_motivational(label):

  if label == 'not_motivational':
    return 0
    
  else:
    return 1



In [ ]:
df_sentence = df_merged.copy()

df_sentence['humour'] = df_sentence['humour'].apply(apply_humour)
df_sentence['sarcasm'] = df_sentence['sarcasm'].apply(apply_sarcasm)
df_sentence['offensive'] = df_sentence['offensive'].apply(apply_offensive)
df_sentence['motivational'] = df_sentence['motivational'].apply(apply_motivational)

In [ ]:
df_sentence.shape

In [ ]:
df_sentence.tail()

In [ ]:


humour_labels = []
sarcasm_labels = []
offensive_labels = []
motivational_labels = []

for img_name in df_word_cleaned['image_name']:

  labels = df_sentence[df_sentence['image_name'] == img_name][['humour','sarcasm','offensive','motivational']]

  humour_labels.append(labels['humour'].values[0])
  sarcasm_labels.append(labels['sarcasm'].values[0])
  offensive_labels.append(labels['offensive'].values[0])
  motivational_labels.append(labels['motivational'].values[0])


  # if labels.shape[0]!=1:
  #   print(img_name)

In [ ]:
humour_series = pd.Series(humour_labels, dtype=int)
sarcasm_series = pd.Series(sarcasm_labels, dtype=int)
offensive_series = pd.Series(offensive_labels, dtype=int)
motivational_series = pd.Series(motivational_labels, dtype=int)

In [ ]:
df_word_cleaned['humour'] = humour_series
df_word_cleaned['sarcasm'] = sarcasm_series
df_word_cleaned['offensive'] = offensive_series
df_word_cleaned['motivational'] = motivational_series

In [ ]:
df_train, df_val = train_test_split(df_word_cleaned,test_size=0.2)
df_test,df_val = train_test_split(df_val,test_size=0.5)

### Dataset

In [ ]:
with open('./memotion_img_sentence_encodings.pkl','rb') as f:
  img_embed_pkl = pickle.load(f)


In [ ]:
def ret_img_word_embeddings(images):

  img_embeddings = []
  word_embeddings = []

  for x in images:

    # img_name, word = df_train.iloc[i]['image_name'], df_train.iloc[i]['word']

    img_embeddings.append(img_embed_pkl[x][0])
    word_embeddings.append(img_embed_pkl[x][1])


  np_img_embeddings_train = np.array(img_embeddings)
  np_word_embeddings_train = np.array(word_embeddings)

  return np_img_embeddings_train, np_word_embeddings_train

# labels_temp_train = df_train.dropna()[['humour','sarcasm','offensive','motivational']].values

In [ ]:

# df = pd.read_csv('./drive/MyDrive/Wipro/Data/memotion_words.csv')

df = pd.read_csv('/content/drive/MyDrive/Wipro/Final/Data/Memotion/csv/memotion_train_sentences.csv')
# df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,look there my friend lightyear now all sohalik...,3,1,0,0,2
1,image_2.jpeg,the best of #10 yearchallenge! completed in le...,0,1,0,1,2
2,image_3.JPG,sam thorne @strippin ( follow follow saw every...,2,0,0,0,2
3,image_4.png,10 year challenge - sweet dee edition,2,2,2,1,2
4,image_5.png,10 year challenge with no filter 47 hilarious ...,3,3,2,0,1


In [ ]:

# Remove Images that don't exist

rem_images = ['image_1567.jpg','image_4924.jpg','image_5119.png','image_6357.jpg']
df_word_cleaned = df.drop(df[df['image_name'].isin(rem_images)].index)
df_word_cleaned.dropna(inplace=True)
df_word_cleaned.index = range(df_word_cleaned.shape[0])


In [ ]:
df_word_cleaned.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,look there my friend lightyear now all sohalik...,3,1,0,0,2
1,image_2.jpeg,the best of #10 yearchallenge! completed in le...,0,1,0,1,2
2,image_3.JPG,sam thorne @strippin ( follow follow saw every...,2,0,0,0,2
3,image_4.png,10 year challenge - sweet dee edition,2,2,2,1,2
4,image_5.png,10 year challenge with no filter 47 hilarious ...,3,3,2,0,1


In [ ]:
def apply_labels(x):
  
  if x>=1:
    return 1

  else:
    return 0

In [ ]:
df_word_cleaned['humour'] = df_word_cleaned['humour'].apply(apply_labels)
df_word_cleaned['sarcasm'] = df_word_cleaned['sarcasm'].apply(apply_labels)
df_word_cleaned['offensive'] = df_word_cleaned['offensive'].apply(apply_labels)

In [ ]:
def ret_dataset(df):

  dataset = tf.data.Dataset.from_tensor_slices(
    (
        df['image_name'].values,
        df['text_corrected'].values,
        df['humour'].values,
        df['sarcasm'].values,
        df['offensive'].values,
        df['motivational'].values,
    )  
  )

  return dataset


In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=21)

In [ ]:

class_weights_h = list(compute_class_weight('balanced', np.unique(df_word_cleaned['humour'].values), df_word_cleaned['humour'].values))
class_weights_s = list(compute_class_weight('balanced', np.unique(df_word_cleaned['sarcasm'].values), df_word_cleaned['sarcasm'].values))
class_weights_o = list(compute_class_weight('balanced', np.unique(df_word_cleaned['offensive'].values), df_word_cleaned['offensive'].values))
class_weights_m = list(compute_class_weight('balanced', np.unique(df_word_cleaned['motivational'].values), df_word_cleaned['motivational'].values))

In [ ]:
class_weights_h, class_weights_s, class_weights_o, class_weights_m

([2.1173438447543966, 0.6545744281964755],
 [2.262799740764744, 0.6418198529411765],
 [1.289327917282127, 0.8167251461988304],
 [0.7726266873202036, 1.41700487012987])

In [ ]:
# class_weights_dict = df_train['humour'].value_counts().to_dict()
class_weights_dict_h = { k : v for k,v in enumerate(class_weights_h)}
class_weights_dict_s = { k : v for k,v in enumerate(class_weights_s)}
class_weights_dict_o = { k : v for k,v in enumerate(class_weights_o)}
class_weights_dict_m = { k : v for k,v in enumerate(class_weights_m)}




In [ ]:
class_weights_dict_m

{0: 0.7726266873202036, 1: 1.41700487012987}

## MTL Model


In [ ]:
def get_mtl_model(base_model):

  base_model.trainable = False

  inp1 = base_model.layers[0].input
  inp2 = base_model.layers[1].input

  img_embedding = base_model.layers[-2].output
  text_embedding = base_model.layers[-1].output

  concat_x = tf.keras.layers.Concatenate()([img_embedding,text_embedding])
  temp_layer = tf.keras.layers.Dense(256,activation='relu',name='first_hidden_layer', kernel_regularizer=tf.keras.regularizers.L2(1e-2))(concat_x)
  temp_layer = tf.keras.layers.BatchNormalization(name='batch_norm_mtl')(temp_layer)
  temp_layer = tf.keras.layers.Dropout(0.5, name='mtl_dropout')(temp_layer)

  out1 = tf.keras.layers.Dense(1, name='out_1')(temp_layer)
  out2 = tf.keras.layers.Dense(1, name='out_2')(temp_layer)
  out3 = tf.keras.layers.Dense(1, name='out_3')(temp_layer)
  out4 = tf.keras.layers.Dense(1, name='out_4')(temp_layer)

  mtl_model = tf.keras.Model(inputs=[inp1,inp2],outputs= [out1, out2, out3, out4])

  return mtl_model

## Training

In [ ]:


humour_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
sarcasm_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
offensive_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
motivational_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)



In [ ]:
base_model = K.models.load_model('./drive/MyDrive/Wipro/Final/Base_Models/models/stanford_scratch_combined_memotion_sentences_basic_model.hdf5')

In [ ]:
base_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_text (InputLayer)         [(None, 300)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 4096)         16781312    input_image[0][0]                
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 4096)         1232896     input_text[0][0]                 
______________________________________________________________________________________________

In [ ]:
mtl_model = get_mtl_model(base_model)

In [ ]:
mtl_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_text (InputLayer)         [(None, 300)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 4096)         16781312    input_image[0][0]                
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 4096)         1232896     input_text[0][0]                 
______________________________________________________________________________________________

In [ ]:
def eval_metrics(dataset, model):



  for x in dataset.batch(len(dataset)):

    img_names = [c.decode('utf-8') for c in x[0].numpy()]
    word_names = [c.decode('utf-8') for c in x[1].numpy()]

    np_imgs, np_words = ret_img_word_embeddings(img_names)
    np_h = tf.expand_dims(x[2],-1)
    np_s = tf.expand_dims(x[3],-1)
    np_o = tf.expand_dims(x[4],-1)
    np_m = tf.expand_dims(x[5],-1)

    out = model.predict([tf.squeeze(tf.convert_to_tensor(np_imgs), 1), tf.squeeze(tf.convert_to_tensor(np_words), 1)])

    # humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out,axis=-1).numpy()), average='macro' )

    

    humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(((tf.round(tf.nn.sigmoid(out[0]).numpy()))[:,0]).numpy()), average='macro' )
    sarcasm_metric = f1_score(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()), list(((tf.round(tf.nn.sigmoid(out[1]).numpy()))[:,0]).numpy()), average='macro' )
    offensive_metric = f1_score(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()), list(((tf.round(tf.nn.sigmoid(out[2]).numpy()))[:,0]).numpy()), average='macro' )
    motivational_metric = f1_score(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()), list(((tf.round(tf.nn.sigmoid(out[3]).numpy()))[:,0]).numpy()), average='macro')

    print(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()))
    print(list(((tf.round(tf.nn.sigmoid(out[0]).numpy()))[:,0]).numpy()))

    print(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()))
    print(list(((tf.round(tf.nn.sigmoid(out[1]).numpy()))[:,0]).numpy()))
    
    print(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()))
    print(list(((tf.round(tf.nn.sigmoid(out[2]).numpy()))[:,0]).numpy()))

    print(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()))
    print(list(((tf.round(tf.nn.sigmoid(out[3]).numpy()))[:,0]).numpy()))

    return humour_metric, sarcasm_metric, offensive_metric, motivational_metric


In [ ]:
def get_sample_weight(class_weight, y_train, motivation=False):

  sample_weight = np.ones(shape=(len(y_train),))
  sample_weight[y_train == 0] = class_weight[0]
  sample_weight[y_train == 1] = class_weight[1]

  if motivation:
    return sample_weight

  sample_weight[y_train == 2] = class_weight[2]
  sample_weight[y_train == 3] = class_weight[3]

  return sample_weight

In [ ]:
humour_metric_val_values = []
sarcasm_metric_val_values = []
offensive_metric_val_values = []
motivational_metric_val_values = []
average_metric_val_values = []

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
epochs = 75
best_val_acc = 0
cur_split = 1

for train_idx, test_idx in kfold.split(df_word_cleaned):
  # df_train = df_word_cleaned.iloc
  # print(train_idx.shape, test_idx.shape)
  mtl_model = get_mtl_model(base_model)
  
  print(f'\n\n----------------------------------------TRAINING SPLIT {cur_split}----------------------------------------')
  cur_split+=1

  df_train = df_word_cleaned.iloc[train_idx]
  df_val = df_word_cleaned.iloc[test_idx]

  # train_dataset  = tf.data.Dataset.from_tensor_slices(
  #     (
  #         df_train['image_name'].values,
  #         df_train['word'].values,
  #         df_train['humour'].values,
  #         df_train['sarcasm'].values,
  #         df_train['offensive'].values,
  #         df_train['motivational'].values,
  #     )  
  #   )



  train_dataset = ret_dataset(df_train)
  val_dataset = ret_dataset(df_val)


  for epoch in range(epochs):

    print(f'Epoch {epoch+1}/{epochs} ',end=' ')

    losses = []


    for x in train_dataset.shuffle(38000).batch(64, drop_remainder=True):

      img_names = [c.decode('utf-8') for c in x[0].numpy()]
      word_names = [c.decode('utf-8') for c in x[1].numpy()]

      np_imgs, np_words = ret_img_word_embeddings(img_names)
      np_h = tf.expand_dims(x[2],-1)
      np_s = tf.expand_dims(x[3],-1)
      np_o = tf.expand_dims(x[4],-1)
      np_m = tf.expand_dims(x[5],-1)


      # sample_weight_h = compute_sample_weight(class_weight=class_weights_dict_h, y=x[2])
      # sample_weight_s = compute_sample_weight(class_weight=class_weights_dict_s, y=x[3])
      # sample_weight_o = compute_sample_weight(class_weight=class_weights_dict_o, y=x[4])
      # sample_weight_m = compute_sample_weight(class_weight=class_weights_dict_m, y=x[5])
      sample_weight_h = get_sample_weight(class_weights_dict_h, x[2], True)
      sample_weight_s = get_sample_weight(class_weights_dict_s, x[3], True)
      sample_weight_o = get_sample_weight(class_weights_dict_o, x[4], True)
      sample_weight_m = get_sample_weight(class_weights_dict_m, x[5], True)



      with tf.GradientTape() as tape:

          out = mtl_model([tf.squeeze(tf.convert_to_tensor(np_imgs), 1), tf.squeeze(tf.convert_to_tensor(np_words), 1)],training=True)

          humour_loss = humour_loss_fn(np_h, out[0], sample_weight= sample_weight_h)
          sarcasm_loss = sarcasm_loss_fn(np_s, out[1], sample_weight= sample_weight_s)
          offensive_loss = offensive_loss_fn(np_o, out[2], sample_weight= sample_weight_o)
          motivational_loss = motivational_loss_fn(np_m, out[3], sample_weight= sample_weight_m)

          loss = humour_loss  + sarcasm_loss + offensive_loss + motivational_loss

          losses.append(loss.numpy())

      gradients = tape.gradient(loss, mtl_model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, mtl_model.trainable_weights))

    humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.round(tf.nn.sigmoid(out[0]).numpy())), average='macro' )
    sarcasm_metric = f1_score(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()), list(tf.round(tf.nn.sigmoid(out[1]).numpy())), average='macro' )
    offensive_metric = f1_score(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()), list(tf.round(tf.nn.sigmoid(out[2]).numpy())), average='macro' )
    motivational_metric = f1_score(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()), list(tf.round(tf.nn.sigmoid(out[3]).numpy())), average='macro')
    average_metric = (humour_metric+ sarcasm_metric+ offensive_metric+ motivational_metric)/4


    
    print(f'Humour F1 score {humour_metric} ', end=' ')
    print(f'Sarcasm F1 score {sarcasm_metric} ---- ', end=' ')
    print(f'Offensive F1 score {offensive_metric} ---- ', end=' ')
    print(f'Motivational F1 score {motivational_metric}', end=' ')
    print(f'Avg F1 score {average_metric}', end=' ')
    print(f'Loss {sum(losses)/len(losses)}',)
    print()


  humour_metric_val, sarcasm_metric_val, offensive_metric_val, motivational_metric_val = eval_metrics(val_dataset, mtl_model)
  average_metric_val = (humour_metric_val+ sarcasm_metric_val+ offensive_metric_val+ motivational_metric_val)/4
  # humour_metric_val = eval_metrics(val_dataset, mtl_model)

  print()

  print(f'Val Humour F1 score {humour_metric_val}', end=' ')
  print(f'Val Sarcasm F1 score {sarcasm_metric_val} ---- ', end=' ')
  print(f'Val Offensive F1 score {offensive_metric_val} ---- ', end=' ')
  print(f'Val Motivational F1 score {motivational_metric_val}')
  print(f'Val Avg F1 score {average_metric_val}', end=' ')

  humour_metric_val_values.append(humour_metric_val)
  sarcasm_metric_val_values.append(sarcasm_metric_val)
  offensive_metric_val_values.append(offensive_metric_val)
  motivational_metric_val_values.append(motivational_metric_val)
  average_metric_val_values.append(average_metric_val)

  # if average_metric_val > best_val_acc:
  #   best_val_acc = average_metric_val
  #   print('\n!!! Saved Model !!!\n')
  #   K.models.save_model(mtl_model, './scratch_flickr30k_memotion_words.hdf5')

    # humour_metric_test, sarcasm_metric_test, offensive_metric_test, motivational_metric_test = eval_metrics(test_dataset, mtl_model)
    # average_metric_test = (humour_metric_test+ sarcasm_metric_test+ offensive_metric_test+ motivational_metric_test)/4

    # print()

    # print(f'Test Humour F1 score {humour_metric_test}')
    # print(f'Test Sarcasm F1 score {sarcasm_metric_test}')
    # print(f'Test Offensive F1 score {offensive_metric_test}')
    # print(f'Test Motivational F1 score {motivational_metric_test}')
    # print(f'Test Avg F1 score {average_metric_test}')
    
    # print()

  # break



----------------------------------------TRAINING SPLIT 1----------------------------------------
Epoch 1/75  Humour F1 score 0.4213381555153707  Sarcasm F1 score 0.3546528377989052 ----  Offensive F1 score 0.46218487394957986 ----  Motivational F1 score 0.5058530510585305 Avg F1 score 0.43600722958059657 Loss 2.868757080757755

Epoch 2/75  Humour F1 score 0.4  Sarcasm F1 score 0.3764102564102564 ----  Offensive F1 score 0.46875 ----  Motivational F1 score 0.5 Avg F1 score 0.4362900641025641 Loss 2.843604394759255

Epoch 3/75  Humour F1 score 0.4345381526104417  Sarcasm F1 score 0.4652837798905215 ----  Offensive F1 score 0.46874999999999994 ----  Motivational F1 score 0.39139139139139134 Avg F1 score 0.4399908309730886 Loss 2.8290339585008293

Epoch 4/75  Humour F1 score 0.43333333333333335  Sarcasm F1 score 0.5477726574500769 ----  Offensive F1 score 0.48051948051948046 ----  Motivational F1 score 0.5730170496664195 Avg F1 score 0.5086606302423275 Loss 2.8193986717311814

Epoch 5/75

In [ ]:
sum(average_metric_val_values)/len(average_metric_val_values)

0.4230924615850678

In [ ]:
K.models.save_model(mtl_model, './drive/MyDrive/Wipro/Final/Task_C/kfold/stanford/models/sentences/task_b/memotion_sentences_stanford_combined_memotion_taskb_best_split.hdf5')